In [45]:
import numpy as np
import json
import os,sys
import pandas as pd
import operator
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
import ast


# train_dir_path='/home/irlplab/Documents/Share/Akansha/GoogleNLQ/natural_questions/v1.0/train/'
# test_dir_path='/home/irlplab/Documents/Share/Akansha/GoogleNLQ/natural_questions/v1.0/dev/'
test_dir_path='/home/irlplab/Documents/Share/Akansha/temp/test/'
train_dir_path='/home/irlplab/Documents/Share/Akansha/temp/train/'

answer_candidate = []
long_answer={}
train_questions=[]
test_questions=[]
beta = 3
true_pos=0
true_neg=0
recall_den=0
precision_den=0

no_of_annotators = 5

def load_json_multiple(segments):
    chunk = ""
    for segment in segments:
        
        chunk += segment
        try:
            yield json.loads(chunk)
            chunk = ""
        except ValueError:
            pass

def get_predicted_answer(segments,example_id):
#     ex_obj = ast.literal_eval(segments)
#     print(ex_obj)
#     return ex_obj
    for segment in segments:
        
        ex_object = json.loads(segment)
#         print(ex_object['example_id'])
        
        if(ex_object['example_id'] == example_id):
            return ex_object['annotations'][0]['long_answer']
    
        

def get_train_data():
    json_files = [pos_json for pos_json in os.listdir(train_dir_path) if pos_json.endswith('.jsonl')]
    all_data=[]
      
    for index, js in enumerate(json_files):
        with open(os.path.join(train_dir_path, js)) as json_file:
            for parsed_json in load_json_multiple(json_file):
                data = {}
                data["example_id"]=parsed_json['example_id']
                data["question_text"]=parsed_json['question_text']
                data["file_name"]=js
                data["document_url"]=parsed_json['document_url']
                all_data.append(data)

    return all_data

def get_test_data():
    json_files = [pos_json for pos_json in os.listdir(test_dir_path) if pos_json.endswith('.jsonl')]
    all_data=[]
#     print(json_files)
     
    for index, js in enumerate(json_files):
        with open(os.path.join(test_dir_path, js)) as json_file:
            for parsed_json in load_json_multiple(json_file):
                
                data = {}
                data["example_id"]=parsed_json['example_id']
                data["question_text"]=parsed_json['question_text']
                data["file_name"]=js
                data["annotations"]=parsed_json['annotations']
                data["document_url"]=parsed_json['document_url']
                all_data.append(data)
                
               
    return all_data


def get_closest_document_index(train_vector,query_vector):
    
    cosine_similarities = linear_kernel(query_vector, train_vector).flatten()
    index, value = max(enumerate(cosine_similarities), key=operator.itemgetter(1))
    print("index;; "+str(index))
    example_id = train_questions[index]['example_id']
    f_name = train_questions[index]['file_name']
    doc_url = train_questions[index]['document_url']
    return example_id,f_name,doc_url

    

#check if long answer is null or not
def f_predicted_l():
    candidate_index=long_answer["candidate_index"]
    if(candidate_index==-1):
        return 0
    else:
        return 1
      

def get_annotator_answer(idx):
    annotator_answer = []
    test_obj = test_questions[idx]['annotations']
    for i in range(no_of_annotators):
        annotator_answer.append(test_obj[i]['long_answer'])
    return annotator_answer
            
    
    
def h_function(predicted_answer,annotator_answer):
    match_flag = 0
    g_count = 0
    global true_pos
    global true_neg
    global recall_den
    global precision_den
    print(predicted_answer)
    print (annotator_answer)
    for i in range(no_of_annotators):
        if(annotator_answer[i]['candidate_index'] != -1):
            g_count = g_count+1
    
    if(g_count>=beta):
        recall_den=recall_den+1
    
    if(predicted_answer['candidate_index']!=-1):
        precision_den=precision_den+1
    
    if(predicted_answer['candidate_index']!=-1 and g_count >=beta):
        for i in range(no_of_annotators):               
            if(annotator_answer[i]['candidate_index'] == predicted_answer['candidate_index'] and \
               annotator_answer[i]['end_byte'] == predicted_answer['end_byte'] and \
               annotator_answer[i]['end_token'] == predicted_answer['end_token'] and \
               annotator_answer[i]['start_byte'] == predicted_answer['start_byte'] and\
               annotator_answer[i]['start_token'] == predicted_answer['start_token']):
                match_flag=1
                true_pos=true_pos+1
                print("first")
                return 1
        if(match_flag==0):
            print("second")
            return 0
            
    elif(g_count < beta and predicted_answer['candidate_index']==-1):
        true_neg=true_neg+1
        print("third")
        return 1
    
    else:
        print("fourth")
        return 0
        

def f1_score():
    recall=true_pos/recall_den
    
    print("recall "+ str(recall))
    
    precision=true_pos/precision_den
    
    print("Precision "+ str(precision))
    
    if(precision!=0 and recall!=0):
        f1=2*(precision*recall)/(precision+recall)
    else:
        f1=0
    print("F1 Score "+ str(f1))
       
    print("true_pos "+str(true_pos))
    print("recall_den "+str(recall_den))
    print("precision_den "+str(precision_den))

    
def get_long_ans_id():
    all_questions=[]
    l_candidate=[]
    for idx, train_question in enumerate(train_questions):
        all_questions.append(train_question["question_text"])
    vectorizer = TfidfVectorizer(stop_words=None)
    train_vector = vectorizer.fit_transform(all_questions)
    for idx,test_question in enumerate(test_questions):
        query=test_question['question_text']
        query_vector = vectorizer.transform([query])
        e_id,f_name,doc_url=get_closest_document_index(train_vector,query_vector)
        data={}
        data['example_id']=e_id
        data['file_name']=f_name
        data['document_url']=doc_url
        l_candidate.append(data)
    return l_candidate
 
def get_long_ans(l_candidates):
    for idx,l_candidate in enumerate(l_candidates):
        pred_url=l_candidate['document_url']
        actual_url=test_questions[idx]['document_url']
        print("pred url"+pred_url)
        print("actual url"+actual_url)
        if(pred_url==actual_url):
            print("inside if")
            #get long answer from example_id->filename
            #compare it with actual answers
            file_name=l_candidate['file_name']
            with open(os.path.join(train_dir_path, file_name)) as json_file:
                predicted_answer = get_predicted_answer(json_file,l_candidate['example_id'])
            annotator_answer = get_annotator_answer(idx)
            h_value = h_function(predicted_answer,annotator_answer)
            print("h value = "+ str(h_value))
            
        else:
            #SET LONG ANSWER AS NULL
            print ("Mis-matched url")
        
def main():
       
    print ("Start time")
    currentDT1 = datetime.datetime.now()
    print (str(currentDT1))

    global train_questions
    train_questions=get_train_data()
    print(train_questions)
    
    global test_questions
    test_questions=get_test_data()
    
    print(test_questions)
    
    l_candidate = get_long_ans_id()
  
    
    get_long_ans(l_candidate)
    
    f1_score()

    print ("End time")
    currentDT2 = datetime.datetime.now()
    print (str(currentDT2))

if __name__ == '__main__':
    main()



    
    


Start time
2019-02-26 23:05:44.847299
[{'example_id': 4549465242785278785, 'question_text': 'when is the last episode of season 8 of the walking dead', 'file_name': 'nq-train-sample.jsonl', 'document_url': 'https://en.wikipedia.org//w/index.php?title=The_Walking_Dead_(season_8)&amp;oldid=828222625'}, {'example_id': -2543388002166163252, 'question_text': 'in greek mythology who was the goddess of spring growth', 'file_name': 'nq-train-sample.jsonl', 'document_url': 'https://en.wikipedia.org//w/index.php?title=Persephone&amp;oldid=853636674'}, {'example_id': 5985355041383167183, 'question_text': 'benefits of colonial life for single celled organisms', 'file_name': 'nq-train-sample.jsonl', 'document_url': 'https://en.wikipedia.org//w/index.php?title=Colony_(biology)&amp;oldid=835462497'}, {'example_id': -2975172535563055798, 'question_text': 'how many season of the man in the high castle', 'file_name': 'nq-train-sample.jsonl', 'document_url': 'https://en.wikipedia.org//w/index.php?title=T

{'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}
[{'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}]
third
h value = 1
recall 1.0
Precision 1.0
F1 Score 1.0
true_pos 3
recall_den 3
precision_den 3
End time
2019-02-26 23:05:46.921823
